In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("joint1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("joint2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2

In [6]:
x = MechanismState(Float64, mechanism);

In [7]:
rand!(x)
setdirty!(x)
configuration(x, joint1)

7-element Array{Float64,1}:
 -0.649067
 -0.718962
  0.177398
 -0.174173
  0.707608
  0.687687
  0.512247

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
relative_transform(x, joint2.frameAfter, body1.frame)

Transform3D from "after_joint2" to "body1":
rotation: 2.2518226404822252 rad about [0.9463305913745417,0.08642649284987565,-0.3114303664744036], translation: Vec(0.705934257385864,0.41318446835276856,0.25430162729150907)


In [11]:
center_of_mass(x)

Point3D in "world": Vec(0.608547869672064,1.6602829217772035,-0.8406660653780158)

In [12]:
mass(mechanism)

0.7974536341598837

In [13]:
transform(x, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(0.7389130487812459,1.60426864017217,-1.0058493870880585)

In [14]:
relative_twist(x, body2, body1)

Twist of "after_joint2" w.r.t "after_joint1" in "world":
angular: Vec(0.009102873162291747,-0.07130901606952256,-0.03457078917342743), linear: Vec(-0.12718696304431498,0.016388687835879434,-0.06729461641925127)

In [15]:
relative_twist(x, body1, world) + relative_twist(x, body2, body1) + relative_twist(x, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [16]:
p = path(mechanism, body2, world)
J = geometric_jacobian(x, p)

GeometricJacobian: body: "world", base: "after_joint2", expressed in "world":
6x7 Array{Float64,2}:
 -0.115791  -0.844872    0.494699   …  -0.0       -0.0       -0.0     
  0.941959  -0.14539    -0.578638      -0.0       -0.0       -0.0     
  0.458187   0.514833    0.648422      -0.0       -0.0       -0.0     
  1.68253    0.0516473  -0.0100116     -0.844872   0.494699  -0.203628
 -0.222092  -0.178512   -0.520654      -0.14539   -0.578638  -0.802521
  0.881786   0.034344   -0.456982   …   0.514833   0.648422  -0.560799

In [17]:
Twist(J, velocity_vector(x, p))

Twist of "world" w.r.t "after_joint2" in "world":
angular: Vec(-0.6959088083619897,-0.5939198971187873,0.14010611650342936), linear: Vec(-0.5057833454280992,-0.6099486713328761,0.5795212753101049)

In [18]:
mass_matrix(x)

7x7 Symmetric{Float64,Array{Float64,2}}:
  2.10258      -0.528904     -0.282463     …   1.05135       0.0435898
 -0.528904      1.02505      -0.868239        -0.408689     -0.0457307
 -0.282463     -0.868239      1.82218         -5.55112e-17  -0.17765  
  1.07553e-16   0.664677     -1.05135          5.55112e-17   0.0217008
 -0.664677      2.77556e-17   0.408689         5.55112e-17  -0.0607898
  1.05135      -0.408689     -5.55112e-17  …   0.797454      0.0230718
  0.0435898    -0.0457307    -0.17765          0.0230718     0.112903 

In [19]:
A = momentum_matrix(x)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
  2.22475   -1.26259    0.620309   …  -0.470363   1.28071    0.0270521
 -1.00178    0.104751   0.943983       0.646542  -0.408785  -0.199876 
 -0.67931   -1.10516    1.95316        0.935812   0.119954  -0.109476 
  0.542898   0.478346  -1.09043       -0.3945     0.162384   0.0531051
  0.45912   -0.231344   0.0836274      0.461437   0.639973  -0.0135046
  1.02058   -0.57139    0.276265   …  -0.517087   0.447211   0.0411837

In [20]:
Momentum(A, velocity_vector(x))

Momentum expressed in "world":
angular: Vec(1.188031492800057,-0.09671294203992675,0.19483718627046687), linear: Vec(0.2024008375981616,0.8816561795586723,0.1810979356382029)

In [21]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(x, accel, world.frame)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(0.815647470694881,-0.9663154269521315,-0.4834042725633093), linear: Vec(-1.0877871820652403,-1.3091177242856478,-1.6265824197919263)

In [22]:
bias_acceleration(x, body2)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(-0.030514833971429393,0.022795535099675232,-0.05505514590362262), linear: Vec(-0.10010891061175824,0.0768701639776091,0.04532087517580509)

In [23]:
v̇ = rand(num_velocities(mechanism))
τ = inverse_dynamics(x, v̇)

7-element Array{Float64,1}:
 10.3385  
 -5.89146 
  3.07513 
 -4.26739 
 -5.07772 
  4.62104 
  0.381684

In [24]:
relative_acceleration(x, body1, body2, v̇)

SpatialAcceleration of "after_joint1" w.r.t "after_joint2" in "world":
angular: Vec(-0.02074013659658456,0.3941629339572144,0.25787186024455205), linear: Vec(0.8448788256737165,-0.17517929998400725,0.3450019203453941)

In [25]:
rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [26]:
rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
    Vertex: body3, Edge: joint3
    Vertex: body8, Edge: joint8
  Vertex: body4, Edge: joint4
    Vertex: body5, Edge: joint5
    Vertex: body9, Edge: joint9
      Vertex: body10, Edge: joint10
  Vertex: body6, Edge: joint6
    Vertex: body11, Edge: joint11
  Vertex: body7, Edge: joint7